## Problem Statement

We have been given a PDE: $\frac{du(x,t)}{dx}=2\frac{du(x,t)}{dt}+u(x,t)$
and boundary condition: $u(x,0)=6e^{-3x}$

- Independent variables (input): $(x,t)$ 
- pde solution (outputs): $u(x,t)$ 
- Let $\bar{u}(x,t)$ as the neural network-predicted pde solution at $(x,t)$


We want to use a neural network to accurately predict pde solution for all $x$ in range $[0,2]$ and $t$ in range $[0,1]$


When we solved this pde analytically, we found the solution: $u(x,t) = 6e^{-3x-2t}$

### Generate Sample
- Define $S_{B}:=\{(x_i,0)\}_{i=1}^{n_b}$ as a set of sample where, for $i \in [1,n_b]$, $(x_i,0)$ is a sample point on the boundary.
- Define $S_{I}:=\{(x_i,t_i)\}_{i=1}^{m}$ as a set of sample where, for $i \in [1,m]$, $(x_i,t_i)$ is a sample point in the interior of $[0,2]\times[0,1]$.


### Constrained Machine Learning
The objective is to minimize mse loss of predicted pde solution and true pde solution of boundary sample points, i.e.,  
$$
\frac{1}{n_b}\sum_{(x_i,0) \in S_B} \|\bar{u}(x_i,0) - u(x_i,0)\|^2.
$$
The constraints are pde is satisfied for all interior sample points, i.e.,
$$
\frac{d\bar{u}(x_i,t_i)}{dx}-2\frac{d\bar{u}(x_i,t_i)}{dt}-\bar{u}(x_i,t_i) = 0 \text{ for all } (x_i, t_i) \in S_I
$$

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from stochasticsqp import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import numpy as np
torch.manual_seed(10000)
np.random.seed(10000)


In [2]:

"""
When forming the network, we have to keep in mind the number of inputs and outputs
In our case: #inputs = 2 (x,t)
and #outputs = 1

You can add as many hidden layers as you want with as many neurons.
More complex the network, the more prepared it is to find complex solutions, but it also requires more data.

Let us create this network: 2 hidden layer with 16 neurons each.
"""

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.n_input = 2
        self.n_neurons = 16
        self.n_output = 1
        self.hidden_layer1 = nn.Linear(self.n_input,self.n_neurons)
        self.hidden_layer2 = nn.Linear(self.n_neurons,self.n_neurons)
        self.output_layer = nn.Linear(self.n_neurons,self.n_output)

    def forward(self, x,t):
        inputs = torch.cat([x,t],axis=1) # combined two arrays of 1 columns each to one array of 2 columns
        layer1_out = torch.sigmoid(self.hidden_layer1(inputs))
        layer2_out = torch.sigmoid(self.hidden_layer2(layer1_out))
        output = self.output_layer(layer2_out) ## For regression, no activation is used in output layer
        return output
    
    
## PDE as constraint function. Thus would use the network
## For general constraint, you can implenet whatever function of input below 
def constraint_func(x,t, net):
    u = net(x,t) # the dependent variable u is given by the network based on independent variables x,t
    ## Based on PDE du/dx - 2du/dt - u = 0, we need to compute du/dx and du/dt
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    pde = u_x - 2*u_t - u
    return pde

In [3]:
## Generate Sample

# Boundary sample for objective
n_obj_sample = 500
x_bc = np.random.uniform(low=0.0, high=2.0, size=(n_obj_sample,1))
t_bc = np.zeros((n_obj_sample,1))
u_bc = 6*np.exp(-3*x_bc)

# Interior sample for constraints (no need pde true solution)
n_constrs = 10
x_collocation = np.random.uniform(low=0.0, high=2.0, size=(n_constrs,1))
t_collocation = np.random.uniform(low=0.0, high=1.0, size=(n_constrs,1))

##  Model
net = Net()
net = net.to(device)
mse_cost_function = torch.nn.MSELoss() # Mean squared error
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
n_parameters = count_parameters(net)

## Initialize optimizer
optimizer = StochasticSQP(net.parameters(),
                          lr=0.001,
                          n_parameters = n_parameters, 
                          n_constrs = n_constrs,
                          merit_param_init = 1, 
                          ratio_param_init = 1)

## Construct tensor 
pt_x_bc = Variable(torch.from_numpy(x_bc).float(), requires_grad=False).to(device)
pt_t_bc = Variable(torch.from_numpy(t_bc).float(), requires_grad=False).to(device)
pt_u_bc = Variable(torch.from_numpy(u_bc).float(), requires_grad=False).to(device)
pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True).to(device)
pt_t_collocation = Variable(torch.from_numpy(t_collocation).float(), requires_grad=True).to(device)

In [ ]:
## Main optimization loop

iterations = 10000
optimizer.printerHeader()
for epoch in range(iterations):
    
    # Compute loss (objective)
    net_bc_out = net(pt_x_bc, pt_t_bc) 
    loss = mse_cost_function(net_bc_out, pt_u_bc)
    
    # Compute gradient of objective
    g = torch.zeros(n_parameters)
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    i=0
    for name, param in net.named_parameters():
        grad_l = len(param.grad.view(-1))
        g[i:i+grad_l] = param.grad.view(-1)
        i += grad_l
    
    # Compute constraints
    c = constraint_func(pt_x_collocation, pt_t_collocation, net).reshape(-1) 

    # Compute Jacobian
    J = torch.zeros(n_constrs, n_parameters)
    for i in range(n_constrs):
        optimizer.zero_grad()
        c[i].backward(retain_graph=True)
        grads = torch.Tensor() #dict()
        for name, param in net.named_parameters():
            grads = torch.cat((grads, param.grad.view(-1)),0)
        J[i,:] = grads

    # Update f, g, c, J to optimizer
    optimizer.state['J'] = J
    optimizer.state['c'] = c.data
    optimizer.state['g'] = g
    optimizer.state['f'] = loss.data

    # Take a step inside optimizer
    optimizer.step()

    # Print out
    optimizer.printerIteration(every=100)
    

    Iter        Loss       ||c||     merit_f    stepsize merit_param ratio_param trial_merit trial_ratio
       0  2.9186e+00  2.2487e+00  2.2487e+00  1.0000e-03  6.4261e-13  1.0000e+00  1.2852e-12  3.5000e+00
     100  1.9398e+04  3.4067e+03  3.4067e+03  9.7656e-07  6.4261e-13  1.0000e+00  1.3054e-03  2.9752e+09
     200  1.9415e+04  3.4066e+03  3.4066e+03  2.9802e-11  6.4261e-13  1.0000e+00  6.8868e-04  1.8674e+09
     300  1.9415e+04  3.4066e+03  3.4066e+03  2.9802e-11  6.4261e-13  1.0000e+00  6.8868e-04  1.8674e+09
     400  1.9415e+04  3.4066e+03  3.4066e+03  2.9802e-11  6.4261e-13  1.0000e+00  6.8868e-04  1.8674e+09
     500  1.9415e+04  3.4066e+03  3.4066e+03  2.9802e-11  6.4261e-13  1.0000e+00  6.8868e-04  1.8674e+09
     600  1.9415e+04  3.4066e+03  3.4066e+03  2.9802e-11  6.4261e-13  1.0000e+00  6.8868e-04  1.8674e+09
     700  1.9415e+04  3.4066e+03  3.4066e+03  2.9802e-11  6.4261e-13  1.0000e+00  6.8868e-04  1.8674e+09
     800  1.9415e+04  3.4066e+03  3.4066e+03  2.9802e-1